# String similarity on lecture data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# path to the csv file
csvpath = "/content/drive/MyDrive/Co_op_JohnLiu/data/mitlecture4.csv"

## Import libraries

In [ ]:
from difflib import SequenceMatcher
from collections import Counter
import string
import re
from typing import List, Dict, Set

import pandas as pd

## Text Processing and Matching

In [ ]:
class BasicTextSimilarityAnalyzer:
    def __init__(self):
        """Initialize the analyzer with basic stop words"""
        # Common English stop words
        self.stop_words = {
            'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you',
            "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself',
            'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her',
            'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them',
            'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom',
            'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are',
            'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having',
            'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if',
            'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for',
            'with', 'about', 'against', 'between', 'into', 'through', 'during',
            'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down',
            'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',
            'then', 'once'
        }

    def preprocess_text(self, text: str) -> List[str]:
        """
        Preprocess text by:
        1. Converting to lowercase
        2. Removing punctuation
        3. Removing stop words
        4. Splitting into words
        """
        # Convert to lowercase
        text = text.lower()

        # Remove punctuation
        text = text.translate(str.maketrans('', '', string.punctuation))

        # Split into words and remove stop words
        words = [
            word for word in text.split()
            if word not in self.stop_words and len(word) > 1
        ]

        return words

    def sequence_similarity(self, text1: str, text2: str) -> float:
        """Calculate similarity using SequenceMatcher"""
        return SequenceMatcher(None, text1, text2).ratio()

    def word_overlap_similarity(self, text1: str, text2: str) -> float:
        """Calculate word overlap similarity"""
        words1 = set(self.preprocess_text(text1))
        words2 = set(self.preprocess_text(text2))

        intersection = len(words1.intersection(words2))
        union = len(words1.union(words2))

        return intersection / union if union > 0 else 0

    def frequency_similarity(self, text1: str, text2: str) -> float:
        """Calculate similarity based on word frequency patterns"""
        words1 = self.preprocess_text(text1)
        words2 = self.preprocess_text(text2)

        # Create word frequency dictionaries
        freq1 = Counter(words1)
        freq2 = Counter(words2)

        # Get all unique words
        all_words = set(freq1.keys()).union(set(freq2.keys()))

        # Calculate frequency similarity
        similarity = 0
        total = 0

        for word in all_words:
            f1 = freq1.get(word, 0)
            f2 = freq2.get(word, 0)
            # Add similarity contribution based on minimum frequency
            similarity += min(f1, f2)
            total += max(f1, f2)

        return similarity / total if total > 0 else 0

    def find_common_phrases(self, text1: str, text2: str, min_length: int = 3) -> List[str]:
        """Find common phrases between two texts"""
        words1 = self.preprocess_text(text1)
        words2 = self.preprocess_text(text2)

        common_phrases = []

        # Look for common sequences of words
        for i in range(len(words1) - min_length + 1):
            phrase = words1[i:i + min_length]
            phrase_str = ' '.join(phrase)

            # Check if this phrase appears in the second text
            words2_str = ' '.join(words2)
            if phrase_str in words2_str and phrase_str not in common_phrases:
                common_phrases.append(phrase_str)

        return common_phrases[:5]  # Return top 5 common phrases


## Compute Similarity

In [ ]:
lectures = pd.read_csv(csvpath)

In [ ]:
lectures.head()

,URL,Time,Transcript_Text,Transcript_Key_Points,Lecture_Notes_Text,Lecture_Notes_Key_Points,Coverage,Missing_Points
0,https://ocw.mit.edu/courses/6-0001-introductio...,0:00-3:00,All right everyone let's get started. All righ...,Recap of Previous Lecture:\n String Manipul...,HOW DO WE WRITE CODE?\n - so far…\n - co...,Current Coding Approach:\n Scope of Coverag...,Similarity Percentage: 85%\nCovered Language M...,High-Level Context:\nThe transcript mentions a...
1,https://ocw.mit.edu/courses/6-0001-introductio...,3:00-6:00,So think about if you want to use a for loop i...,Functions and Programming Style:\n Reuse an...,GOOD PROGRAMMING\n - more code not necessar...,Good Programming Practices:\n Code Quantity...,Breakdown:\nKey Overlapping Points:\nBoth emph...,Projector Analogy: Demonstrates abstraction in...
2,https://ocw.mit.edu/courses/6-0001-introductio...,6:00-9:00,OK that's abstraction. The other half of that ...,Concept Explanation:\n Breaking a large pro...,EXAMPLE - PROJECTOR\n - a projector is a bl...,Abstraction Example - Projector:\n A projec...,Similarity Score: 85%\nThe transcript and lect...,Specificity in Examples:\nThe transcript provi...
3,https://ocw.mit.edu/courses/6-0001-introductio...,9:00-12:00,OK so decomposition is creating structure in y...,Decomposition and Abstraction:\n Decomposit...,SUPRESS DETAILS with ABSTRACTION\n - in pro...,Abstraction: Suppress Details:\n Projector ...,Similarity Score: 85-90%\nThe transcript and l...,Transcript Details:\nProvides a deeper explana...
4,https://ocw.mit.edu/courses/6-0001-introductio...,12:00-15:00,The first is whoops-- the first is going to be...,Defining a Function:\n def Keyword:\n Us...,HOW TO WRITE and CALL/INVOKE A FUNCTION\n d...,Function Definition (def):\n Keyword def:\n...,Similarity Score: 85%\nJustification for the S...,Flexibility of the Body:\nThe transcript expli...


In [ ]:
analyzer = BasicTextSimilarityAnalyzer()

In [ ]:
transcript_text = lectures.iloc[0]['Transcript_Text']
notes_text = lectures.iloc[0]['Lecture_Notes_Text']

In [ ]:
i = 0
print(f"\nSegment {i} ({lectures.iloc[i]['Time']}):")
print("\nText Statistics:")
print(f"- Transcript length: {len(transcript_text)} characters")
print(f"- Notes length: {len(notes_text)} characters")
print(f"- Transcript words: {len(transcript_text.split())}")
print(f"- Notes words: {len(notes_text.split())}")


Segment 0 (0:00-3:00):

Text Statistics:
- Transcript length: 2427 characters
- Notes length: 431 characters
- Transcript words: 424
- Notes words: 76


In [ ]:
# Calculate similarities
sequence_sim = analyzer.sequence_similarity(transcript_text, notes_text)
word_overlap_sim = analyzer.word_overlap_similarity(transcript_text, notes_text)
freq_sim = analyzer.frequency_similarity(transcript_text, notes_text)

print("\nSimilarity Metrics:")
print(f"- Sequence Similarity: {sequence_sim:.4f}")
print(f"- Word Overlap Similarity: {word_overlap_sim:.4f}")
print(f"- Frequency Similarity: {freq_sim:.4f}")


Similarity Metrics:
- Sequence Similarity: 0.0084
- Word Overlap Similarity: 0.0938
- Frequency Similarity: 0.0868


In [ ]:
# Find common phrases
common_phrases = analyzer.find_common_phrases(transcript_text, notes_text)
if common_phrases:
    print("\nCommon Phrases:")
    for phrase in common_phrases:
        print(f"- '{phrase}'")

print("-" * 70)


Common Phrases:
- 'how write code'
- 'write code so'
- 'code so far'
- 'some piece code'
----------------------------------------------------------------------
